# F0[_sall1a_] in _lhx1a_:GFP

In [1]:
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu, kruskal
from skimage.measure import label, regionprops, regionprops_table
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
from tqdm.notebook import tqdm, trange
import time
import sys
import ipywidgets
from plotParams import *
from juanNapariFx import *


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Light',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [2]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

Plotting style is Dark
Assistant skips harvesting pyclesperanto as it's not installed.


/Users/angueyraaristjm/opt/anaconda3/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [3]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Extract layers from z-stacks<a id='zExtract' name='zExtract'>∮</a>
***
[Back to Index](#Index)

In [16]:
# getting list of files
dPath = '/Volumes/zfSSD/LiImaging/A1R/zf/20221209_F0sall1a/'
gene = 'wt'
#### !ls $dPath/*$gene*[002,004].nd2
# fList = !ls $dPath/*$gene*[002,004].nd2
# fList = !ls $dPath/*5dpf*$gene*[002,004]*mip.nd2
fList = !ls $dPath/*3dpf*$gene*[002,004].nd2
fList = list(map(lambda st: str.replace(st, dPath + '/', 'fPath = \"'), fList))
fList = list(map(lambda st: str.replace(st, '.nd2', '\"; gene = \'' + gene + '\'; '), fList))
print(*fList, sep = "\n")

fPath = "20221209_3dpf_lhx1aGFP_wt_L01_002"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L01_004"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L02_002"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L02_004"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L03_002"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L03_004"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L04_002"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L04_004"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L05_002"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L05_004"; gene = 'wt'; 
fPath = "20221209_3dpf_lhx1aGFP_wt_L06_002"; gene = 'wt'; 


In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

In [214]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/F0sall1a/h1G/"

# open file
dPath = '/Volumes/zfSSD/LiImaging/A1R/zf/20221209_F0sall1a/'


# 5 dpf + immuno
### In notes: 1D4 labeling did not work in any sample; sall1a labels OS
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L12_002"; gene = 'wt'; zlims = [17,19]; zlims2 = [30,42]; zlims3 = [8,12];# very dim GFP; sclera+
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L13_002"; gene = 'wt'; zlims = [10,16]; zlims2 = [29,37]; zlims3 = [7,13]; # dim GFP; sclera+
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L14_002"; gene = 'wt'; # side-view; not bad
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L15_002"; gene = 'wt'; zlims = [24,26]; zlims2 = [35,50]; zlims3 = [19,24]; #ok-ish
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L15_004"; gene = 'wt'; zlims = [17,19]; zlims2 = [26,44]; zlims3 = [13,16]; #ok-ish
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L16_002"; gene = 'wt'; zlims = [15,17]; zlims2 = [23,41]; zlims3 = [7,12]; #ok-ish
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L16_004"; gene = 'wt'; # side-view; not bad 
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L17_002"; gene = 'wt'; zlims = [22,25]; zlims2 = [34,51]; zlims3 = [15,21]; # looks difficult to segment
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L17_004"; gene = 'wt';  # side-view; EXAMPLE?

fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L19_002"; gene = 'sall1a'; zlims = [18,21]; zlims2 = [21,36]; zlims3 = [9,13];# dim Sall1 labeling; GFP will be hard to segment
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L19_004"; gene = 'sall1a'; # side-view
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L20_002"; gene = 'sall1a'; zlims = [16,19]; zlims2 = [33,42]; zlims3 = [11,14]; #Sall1+; good GFP
# fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L21_002"; gene = 'sall1a'; zlims = [24,27]; zlims2 = [35,47]; zlims3 = [17,20]; # good GFP; Sall1 seems negative
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L21_004"; gene = 'sall1a'; #side-view; good
# fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002"; gene = 'sall1a'; zlims = [10,13]; zlims2 = [26,41]; zlims3 = [3,11]; # good GFP
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_004"; gene = 'sall1a'; # side-view; Sall1a is dim but there?
# fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L23_002"; gene = 'sall1a'; zlims = [10,12]; zlims2 = [25,36]; zlims3 = [5,10];
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L23_004"; gene = 'sall1a'; #side-view; in all of these, RPE is bright in 647 channel
# fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L24_002"; gene = 'sall1a'; zlims = [16,18]; zlims2 = [34,47]; zlims3 = [9,14];
#### fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L24_004"; gene = 'sall1a'; # side-view

# # 3 dpf
# fPath = "20221209_3dpf_lhx1aGFP_wt_L01_002"; gene = 'wt'; zlims = [16,18]; zlims2 = [25,48]; # curvy; bad for DAPI but good for GFP
# #### fPath = "20221209_3dpf_lhx1aGFP_wt_L01_004"; gene = 'wt'; #side-view; really nice
# fPath = "20221209_3dpf_lhx1aGFP_wt_L02_002"; gene = 'wt'; zlims = [23,26]; zlims2 = [34,52];
# #### fPath = "20221209_3dpf_lhx1aGFP_wt_L02_004"; gene = 'wt'; #side-view; really nice
# fPath = "20221209_3dpf_lhx1aGFP_wt_L03_002"; gene = 'wt'; zlims = [10,15]; zlims2 = [23,48]; # curvy
# #### fPath = "20221209_3dpf_lhx1aGFP_wt_L03_004"; gene = 'wt'; # side-view; DAPI did not penetrate to INL
# fPath = "20221209_3dpf_lhx1aGFP_wt_L04_002"; gene = 'wt'; zlims = [18,22]; zlims2 = [45,57]; # nice
# #### fPath = "20221209_3dpf_lhx1aGFP_wt_L04_004"; gene = 'wt'; # side-view; DAPI did not penetrate to INL
# fPath = "20221209_3dpf_lhx1aGFP_wt_L05_002"; gene = 'wt'; zlims = [25,27]; zlims2 = [45,57];
# #### fPath = "20221209_3dpf_lhx1aGFP_wt_L05_004"; gene = 'wt'; # side-view; better DAPI
# fPath = "20221209_3dpf_lhx1aGFP_wt_L06_002"; gene = 'wt'; zlims = [2,3]; zlims2 = [7,10];

# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L01_002"; gene = 'sall1a'; zlims = [12,15]; zlims2 = [32,48];
# # #### fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L01_004"; gene = 'sall1a'; #side-view; really nice
# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L02_002"; gene = 'sall1a'; zlims = [7,10]; zlims2 = [26,41];
# # #### fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L02_004"; gene = 'sall1a'; #side-view
# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L03_002"; gene = 'sall1a'; zlims = [14,17]; zlims2 = [31,44];
# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L04_002"; gene = 'sall1a'; zlims = [6,11]; zlims2 = [18,33]; # this retina is not layered yet; very different from rest
# # #### fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L04_004"; gene = 'sall1a'; #side-view; layers are there but more immature than expected for this age

# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)
    print('Created output directory')

print('Canvas clear')


# load whole stack to figure out best planes for cellpose
print('Loading files...')
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

# if len(viewer.layers)==2: # no DAPI, GFP and RFP
#     nChannels = 2
#     viewer.layers[0].colormap = 'green'
#     viewer.layers[0].name = 'R'
#     viewer.layers[1].colormap = 'magenta'
#     viewer.layers[1].name = 'U'
if len(viewer.layers)==2: # DAPI and GFP channel (no RFP or Cy5)
    nChannels = 2
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'H'
elif len(viewer.layers)==3: #DAPI
    nChannels = 3
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'H'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = '1D4'
elif len(viewer.layers)==4: #DAPI
    nChannels = 4
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'H'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = '1D4'
    viewer.layers[3].colormap = 'bop orange'
    viewer.layers[3].name = 'sall1a'
    
print('Loaded: ' + fPath)

Canvas clear
Loading files...
Loaded: 20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L20_002


---

In [215]:
# for 3 dpf
# make mips (and remove any previous ones)
# zlims = [2,3]; zlims2 = [7,10];
# if len(viewer.layers)>nChannels:
#     for l in viewer.layers[nChannels:]:
#         viewer.layers.remove(l)

# # for l in [viewer.layers[0],viewer.layers[1],viewer.layers[2],viewer.layers[3]]: # save all channels including immunos
# for l in [viewer.layers['N']]:
#     l.visible = False
#     viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

# for l in [viewer.layers['H']]:
#     l.visible = False
#     viewer.add_image(l.data[zlims2[0]:zlims2[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

# # for 5 dpf
# make mips (and remove any previous ones)
# zlims = [16,18]; zlims2 = [34,47]; zlims3 = [9,14];
if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
        viewer.layers.remove(l)

# for l in [viewer.layers[0],viewer.layers[1],viewer.layers[2],viewer.layers[3]]: # save all channels including immunos
for l in [viewer.layers['N']]:
    l.visible = False
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

for l in [viewer.layers['H']]:
    l.visible = False
    viewer.add_image(l.data[zlims2[0]:zlims2[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
for l in [viewer.layers['sall1a'],viewer.layers['1D4']]:
    l.visible = False
    viewer.add_image(l.data[zlims3[0]:zlims3[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [216]:
# when things look good, save in folder for batch cellpose analysis
l = viewer.layers['H_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['N_mip']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers['sall1a_mip']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers['1D4_mip']; l.save(dOut + l.name + '.tiff')
print('MIP layers saved for ' + fPath)

MIP layers saved for 20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L20_002


### Establish landmarks in whole-eye MIP
- Use eyeMarks layer to mark:
        - Landmark #1: Center of FOV (where zoom in is centered)
        - Landmark #2: Center of Optic Nerve Head 
        - Landmark #3: Dorsal edge of eye
        - Landmark #4: Ventral edge of eye
        - Landmark #5: Temporal edge of eye (strike zone side)
        - Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

In [147]:
# nMIP = 4 * 2
# viewer.layers['H_mip'].scale = [1.5/8,1.5/8]; viewer.layers['H_mip'].translate = [0,+(512-1024*1.5/(8*2))]
# viewer.layers['N_mip'].scale = [1.5/8,1.5/8]; viewer.layers['N_mip'].translate = [0,+(512-1024*1.5/(8*2))]
# viewer.layers['N_mip'].visible = False;
# viewer.layers['sall1a_mip'].scale = [1.5/8,1.5/8]; viewer.layers['sall1a_mip'].translate = [0,+(512-1024*1.5/(8*2))]
# viewer.layers['sall1a_mip'].visible = False;
# viewer.layers['1D4_mip'].scale = [1.5/8,1.5/8]; viewer.layers['1D4_mip'].translate = [0,+(512-1024*1.5/(8*2))]
# viewer.layers['1D4_mip'].visible = False;


nMIP = 2 * 2
viewer.layers['H_mip'].scale = [1.5/8,1.5/8]; viewer.layers['H_mip'].translate = [0,+(512-1024*1.5/(8*2))]
viewer.layers['N_mip'].scale = [1.5/8,1.5/8]; viewer.layers['N_mip'].translate = [0,+(512-1024*1.5/(8*2))]
viewer.layers['N_mip'].visible = False;

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
viewer.open((dPath+mipPath + ".nd2"))

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # DAPI and GFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'gray'
    viewer.layers[nMIP].name = 'eyeN'
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeH'
# if len(viewer.layers)==2+nMIP: # DAPI and RFP channel (single positive larvae with bad immunos)
#     nChannels = 2
#     viewer.layers[nMIP+0].colormap = 'green'
#     viewer.layers[nMIP+0].name = 'eyeUH
#     viewer.layers[nMIP+0].opacity = 0.66
#     viewer.layers[nMIP+1].colormap = 'magenta'
#     viewer.layers[nMIP+1].name = 'eyeL'
elif len(viewer.layers)==3+nMIP: #DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'gray'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeH'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyePNA'
elif len(viewer.layers)==4+nMIP: #DAPI
    nChannels = 4
    viewer.layers[nMIP+0].colormap = 'gray'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+0].visible = False;
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeH'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eye1D4'
    viewer.layers[nMIP+2].visible = False;
    viewer.layers[nMIP+3].colormap = 'bop orange'
    viewer.layers[nMIP+3].name = 'eyeSall1a'
    

print('Loaded: ' + fPath)
eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
    with out:
        l = viewer.layers['eyeMarks']; l.save(dOut + l.name + '.csv');
        print('eyeMarks layers saved for ' + fPath)
    # clear viewer
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!
    
eyeMarksButton.on_click(eyeMClick)

print("Make 6 landmark points then click button to save")

ipywidgets.VBox([eyeMarksButton,out])

Loading eye mip...
Loaded: 20221209_3dpf_lhx1aGFP_wt_L06_002
Make 6 landmark points then click button to save


### [Go back and run next one &uarr;](#zExtract)
---

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

## Run cellpose as batch analysis
Usually good parameters are :
1. xOPS:GFP: diameter = 20; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws1:mCherry: diameter = 40; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws2:mCherry:
1. mws2:GFP:
1. thrb:tdTomato:
    
flow_threshold = 0.6 will give more ROIs

In [54]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

In [217]:
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/F0sall1a/h1G/"


### collect file paths
wt_gene = 'wt';
wt_fPaths = [
# 5 dpf
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L12_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L13_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L15_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L15_004',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L16_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L17_002',
# # 3 dpf
# '20221209_3dpf_lhx1aGFP_wt_L01_002',
# '20221209_3dpf_lhx1aGFP_wt_L02_002',
# '20221209_3dpf_lhx1aGFP_wt_L03_002',
# '20221209_3dpf_lhx1aGFP_wt_L04_002',
# '20221209_3dpf_lhx1aGFP_wt_L05_002',
# '20221209_3dpf_lhx1aGFP_wt_L06_002',
]

# ### collect file paths
cr_gene = 'sall1a';
cr_fPaths = [
# 5 dpf
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L19_002',
'20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L20_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L21_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L23_002',
# '20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L24_002',
# 3 dpf
# '20221209_3dpf_lhx1aGFP_F0sall1a_L01_002',
# '20221209_3dpf_lhx1aGFP_F0sall1a_L02_002',
# '20221209_3dpf_lhx1aGFP_F0sall1a_L03_002',
# '20221209_3dpf_lhx1aGFP_F0sall1a_L04_002',
]


# define cellpose params
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 40,
    'diameterH' : 100,
    'flow_threshold' : 0.5,
    'cellprob_threshold' : 0.0
}

# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

In [218]:
print('Starting analysis:\n')
with tqdm(total=len(wt_fPaths)*1, file=sys.stdout) as progBar:
    for fPath in wt_fPaths:
        print('\n' + fPath)
        print('\t H1s')
        startTime = time.time()
        dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'H_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterH'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
#         dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'L_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)
        
#         dOut = dAnalysis + wt_gene + '/' + fPath + '/' + 'N_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)
    
print('\nFinished cellpose batch analysis')

print('Starting analysis:\n')
with tqdm(total=len(cr_fPaths)*1, file=sys.stdout) as progBar:
    for fPath in cr_fPaths:
        print('\n' + fPath)
        print('\t H1s')
        startTime = time.time()
        dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'H_mip.tiff'
        img = cellpose.io.imread(dOut)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterH'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
        
#         dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'L_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)
        
#         dOut = dAnalysis + cr_gene + '/' + fPath + '/' + 'N_mip.tiff'
#         img = cellpose.io.imread(dOut)
#         masks, flows, styles, diams = model.eval(img, 
#                                                  diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                                  do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                                  flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
#         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
#         endTime = time.time()
#         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
#         progBar.update(1)

        
print('\nFinished cellpose batch analysis')

Starting analysis:



0it [00:00, ?it/s]


Finished cellpose batch analysis
Starting analysis:



  0%|          | 0/1 [00:00<?, ?it/s]


20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L20_002
	 H1s
	 Time elapsed: 20 s


Finished cellpose batch analysis


***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# viewer = napari.Viewer()

In [266]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()


# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/F0sall1a/h1G/"

# open file
dPath = '/Volumes/zfSSD/LiImaging/A1R/zf/20221209_F0sall1a/'


# 5 dpf + immuno
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L12_002"; gene = 'wt'; # very dim GFP; sclera+
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L13_002"; gene = 'wt'; # dim GFP; sclera+
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L15_002"; gene = 'wt'; #ok-ish
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L15_004"; gene = 'wt'; #ok-ish; bad segmentation
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L16_002"; gene = 'wt'; #ok-ish
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_wt_L17_002"; gene = 'wt'; # looks difficult to segment

fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L19_002"; gene = 'sall1a'; # dim Sall1 labeling; GFP will be hard to segment
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L20_002"; gene = 'sall1a'; #Sall1+; good GFP can be EXAMPLE
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L21_002"; gene = 'sall1a'; # good GFP; Sall1 seems negative
fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002"; gene = 'sall1a'; # good GFP

# fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L23_002"; gene = 'sall1a'; 
# fPath = "20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L24_002"; gene = 'sall1a'; 

# # 3 dpf
# segmentation curated for all these
# fPath = "20221209_3dpf_lhx1aGFP_wt_L01_002"; gene = 'wt'; # curvy; bad for DAPI but good for GFP
# fPath = "20221209_3dpf_lhx1aGFP_wt_L02_002"; gene = 'wt'; # done
# fPath = "20221209_3dpf_lhx1aGFP_wt_L03_002"; gene = 'wt'; # curvy
# fPath = "20221209_3dpf_lhx1aGFP_wt_L04_002"; gene = 'wt'; # EXAMPLE
# fPath = "20221209_3dpf_lhx1aGFP_wt_L05_002"; gene = 'wt'; 
# fPath = "20221209_3dpf_lhx1aGFP_wt_L06_002"; gene = 'wt'; # EXAMPLE too although it's lowest count

# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L01_002"; gene = 'sall1a'; 
# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L02_002"; gene = 'sall1a'; 
# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L03_002"; gene = 'sall1a'; 
# fPath = "20221209_3dpf_lhx1aGFP_F0sall1a_L04_002"; gene = 'sall1a'; # this retina is not layered yet; very different from rest


# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
mipO = 0.99
viewer.open(dOut + 'H' + "_mip.tiff", plugin='builtins', colormap = 'gray', blending='additive', opacity=mipO);
viewer.open(dOut + 'N' + "_mip.tiff", plugin='builtins', colormap = 'bop blue', blending='additive', opacity=mipO);

# load segmentation
segData = np.load(dOut + 'H' + "_mip" + "_seg.npy", allow_pickle=True).item()
viewer.add_labels(segData['masks'], name='H_seg',blending='additive');
viewer.layers['H_seg'].preserve_labels = True;
# tweak display to help with segmentation
# viewer.layers['H_mip'].opacity=mipO;viewer.layers['H_mip'].gamma=1.0;viewer.layers['H_mip'].contrast_limits=[150,2000]

viewer.layers['N_mip'].visible = False;

print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'H_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')
print('removeLabel on H_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'H_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle H_seg preserve_labels ("k")')
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'N_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle N_mip visibility ("B")')

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = 'H_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle H_mip visibility ("N")')

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
    """Set the currently selected label to the largest used label plus one."""
    lname = 'H_seg'
    viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in H_seg visibility ("+"|"M")')

Viewer cleared...
Loaded  20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002 !
 Now fix it and save it!
removeLabel on H_seg ("Shift-X")
toggle H_seg preserve_labels ("k")
toggle N_mip visibility ("B")
toggle H_mip visibility ("N")
Add new label in H_seg visibility ("+"|"M")


#### reload z-stack if needed to fix segmentation

In [267]:
# load whole stack to figure out best planes for cellpose
print('Loading files...')
nMips = len(viewer.layers)
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2: # DAPI and GFP channel (no RFP or Cy5)
    nChannels = 2
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'H'
elif len(viewer.layers)==3: #DAPI
    nChannels = 3
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'H'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = '1D4'
elif len(viewer.layers)==4: #DAPI
    nChannels = 4
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 1.00
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'H'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = '1D4'
    viewer.layers[3].colormap = 'bop orange'
    viewer.layers[3].name = 'sall1a'

print('Loaded: ' + fPath)

Loading files...
Loaded: 20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002
Cut!
Cut!
Cut!


---

#### resave curated segmentation after napari-ing around (H_mip only)

In [268]:
baseName = 'H'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

Done with H_seg for 20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002


### Reload curated segmentations after saving

In [269]:
# remove all except mips
# viewer.layers.remove(viewer.layers['Points'])
n_mips = 2;
if len(viewer.layers)>n_mips:
    for l in viewer.layers[n_mips:]:
        viewer.layers.remove(l)

viewer.open(dOut + 'H' + "_seg_curated.tiff", name='H_seg', plugin='builtins',blending='additive');
viewer.layers['H_mip'].visible = True

print(fPath)
nH = len(np.unique(viewer.layers['H_seg'].data))-1
print("Image has {0} H1s".format(nH))


20221209_5dpf_lhx1aGFP_1D4Cy3_sall1a647_F0sall1a_L22_002
Image has 59 H1s


### [Go back and run next one &uarr;](#manualCuration)
---

### reload mip and eye marks in case there are doubts

***
## Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Read csv created during analysis and create bar plot

In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/"
# dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_combinedCounts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
photoreceptors = ['R','U','S','M','L']
# photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.03
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png", bbox_inches = "tight")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg", bbox_inches = "tight")

### Stats

### Read csv created during analysis and create bar plot

In [ ]:
geneList = ['wt','nr2e3']
photoreceptors = ['R','U','S','M','L']

print('{0} vs. {1}:'.format(geneList[0],geneList[1]))

for pr in photoreceptors:
    # get counts for each photoreceptor subtype and exclude NaNs
    wtCount = df[(df['genotype']==(geneList[0]))&(df['excludeFlag']==False)][pr]
    wtCount = wtCount[~np.isnan(wtCount)]
    
    crCount = df[(df['genotype']==(geneList[1]))&(df['excludeFlag']==False)][pr]
    crCount = crCount[~np.isnan(crCount)]
    
    u, p = mannwhitneyu(wtCount, crCount)
    print('\t{0}:'.format(pr))
    print('\t\tU = {0:.3f}, p = {1:.5f}, nEyes: wt = {2:.0f}; cr = {3:.0f}'.format(u,p,len(wtCount),len(crCount)))


In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/m2Gs2C/"
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_m2Gs2C_Counts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
# photoreceptors = ['R','U','S','M','L']
photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.01
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg")